<a href="https://colab.research.google.com/github/VAEs-Tutorial/paper/blob/master/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

Using TensorFlow backend.


In [34]:
batch_size = 100 

original_dim = 784
latent_dim = 2
intermediate_dim = 256
epochs = 50
epsilon_std = 1.0

x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [32]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])#

decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')#

h_decoded = decoder_h(z)#
x_decoded_mean = decoder_mean(h_decoded)#

# instantiate VAE model
vae = Model(x, x_decoded_mean)

# Compute VAE loss
xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) #
vae_loss = K.mean(xent_loss + kl_loss)


vae.add_loss(vae_loss) #

vae.add_metric(original_dim * metrics.binary_crossentropy(x, x_decoded_mean), name='Reconstruction Loss')
vae.add_metric(- 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1), name='KL_Loss')
vae.compile(optimizer='rmsprop') #



/usr/local/lib/python3.6/dist-packages/keras/engine/training_utils.py:819: UserWarning: Output dense_42 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_42.
  'be expecting any data to be passed to {0}.'.format(name))


In [6]:
(x_train, _), (x_test, y_test) = mnist.load_data()


x_train = x_train.astype('float32') / 255.#数据预处理归一化

x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))#

x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [35]:
vae.fit(x_train,#
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        verbose=2,
        validation_data=(x_test, None))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
 - 2s - loss: 156.7451 - Reconstruction Loss: 151.1743 - KL_Loss: 5.5707 - val_loss: 157.4847 - val_Reconstruction Loss: 151.8109 - val_KL_Loss: 5.6738
Epoch 2/50
 - 2s - loss: 156.4396 - Reconstruction Loss: 150.8390 - KL_Loss: 5.6007 - val_loss: 157.4172 - val_Reconstruction Loss: 151.8541 - val_KL_Loss: 5.5630
Epoch 3/50
 - 2s - loss: 156.1200 - Reconstruction Loss: 150.5065 - KL_Loss: 5.6135 - val_loss: 156.7998 - val_Reconstruction Loss: 151.2624 - val_KL_Loss: 5.5374
Epoch 4/50
 - 2s - loss: 155.8161 - Reconstruction Loss: 150.1769 - KL_Loss: 5.6391 - val_loss: 156.7832 - val_Reconstruction Loss: 151.1684 - val_KL_Loss: 5.6147
Epoch 5/50
 - 2s - loss: 155.5395 - Reconstruction Loss: 149.8628 - KL_Loss: 5.6767 - val_loss: 156.7513 - val_Reconstruction Loss: 151.1158 - val_KL_Loss: 5.6355
Epoch 6/50
 - 2s - loss: 155.2667 - Reconstruction Loss: 149.5845 - KL_Loss: 5.6822 - val_loss: 156.3658 - val_Reconstruction Loss: 150